# <center>Day 4: matching models with nontransferable utility</center>
### <center>Alfred Galichon (NYU+ScPo)</center>
## <center>'math+econ+code' masterclass on equilibrium transport and matching models in economics</center>
<center>© 2020-2021 by Alfred Galichon.  Support from  NSF DMS-1716489 and ERC CoG-866274 EQUIPRICE grants is acknowledged.</center>

#### <center>with Python code</center>

**If you reuse material from this masterclass, please cite as:**<br>
Alfred Galichon, 'math+econ+code' masterclass on equilibrium transport and matching models in economics, June 2021. https://github.com/math-econ-code/mec_equil


# References

## Textbooks

* [RS90] Alvin Roth and Marilda Sotomayor (1990). *Two-sided matching*. Econometric Society Monographs, Cambridge University Press.

## Papers

* [GS62] David Gale and Lloyd Shapley (1962). "College Admissions and the Stability of Marriage." *American Mathematical Monthly* 69 (1), pp. 9–14.

* [A00] Hiroyuki Adachi (2000). "On a characterization of stable matchings." *Economics Letters* 68 pp. 43–49.

* [GH19] Alfred Galichon and Yu-Wei Hsieh (2019)."Aggregate stable matching with money burning." SSRN id=2887732. 


# Setting up the model

## Populations

* As yesterday we consider a population of workers $x\in\mathcal{X}$ and firms $y\in\mathcal{Y}$, and assume that $n_x$ is the number of workers of type $x$, while $m_y$ is the number of firms of type $y$.
* We shall sometimes assume that there is one individual per type, that is $n_x = 1$ and $m_y = 1$, in which case we shall talk about *invidividual matching,* and sometimes allow for more than one agent per type, a situation we shall refer to as *aggregate matching*. 

## Preferences

* Assume the wages are set. We describe the preferences as follows:<br>
    $\alpha_{xy}$ = valuation of an $xy$ match by $x$<br>
    $\gamma_{xy}$ = valuation of an $xy$ match by $y$<br>
    If $x$ and $y$ remain unmatched, they get respective utilities $\alpha_{x0}$ and $\gamma_{0y}$.<br>
* As before we denote $\mathcal{X}_0 = \mathcal{X} \cup \{0\}$ and $\mathcal{Y}_0 = \mathcal{Y} \cup \{0\}$.
* Assume strict preferences:<br>
    $\alpha_{xy} \neq \alpha_{xy^\prime}$ for $y\neq y^\prime$<br>
    $\gamma_{xy} \neq \gamma_{x^\prime y}$ for $x \neq x^\prime$.

* The preferences are usually described in an ordinal way, by specifying a order relation for each agent on the other side of the market.


## Representing the model

Let's first load the libraries we will need.

In [1]:
import numpy as np
import networkx as nx

We create the `NTU_market` class to encompass that information and store it in a convenient way:

In [2]:
class NTU_market:
    def __init__(self,α_x_y,γ_x_y,n_x = np.array([]), m_y = np.array([])):
        nbx = α_x_y.shape[0]
        nby = α_x_y.shape[1]
        self.α_x_y = np.hstack((α_x_y,np.zeros((nbx,1) )))
        self.γ_x_y = np.vstack((γ_x_y,np.zeros( (1,nby) )))
        if n_x.size == 0:
            n_x = np.ones(nbx)
        if m_y.size ==  0:
            m_y = np.ones(nby) 
        self.n_x,self.m_y = n_x, m_y
        self.largex,self.largey = nby+1, nbx+1
        self.smallx,self.smally = -1, -1
        self.nbx,self.nby = nbx, nby
        self.αo_x_y = np.zeros((nbx,nby+1), dtype = 'int64') 
        self.γo_x_y = np.zeros((nbx+1,nby), dtype = 'int64') 
        self.prefslistα_x_y = np.zeros((nbx,nby+1), dtype = 'int64') 
        self.prefslistγ_x_y = np.zeros((nbx+1,nby), dtype = 'int64') 
        self.traceu0_x_t = np.array([])

        for x in range(nbx):
            thelistx = (- self.α_x_y )[x,:].argsort()
            self.αo_x_y[x, thelistx] =  nby  - np.arange(nby+1)
            self.prefslistα_x_y[x,:] = (thelistx+1) % (self.nby+1) - 1
        for y in range(nby):
            thelisty = ( - self.γ_x_y)[:,y].argsort()
            self.γo_x_y[thelisty, y] = nbx  - np.arange(nbx+1)
            self.prefslistγ_x_y[:,y] = (thelisty+1) % (self.nbx+1) - 1
        self.comp_nbsteps = -1
        self.comp_time = -1
        self.eq_μ_x_y = np.array([])                

The following `print_prefs` method prints the ordinal preferences of the agents:

In [3]:
def list_prefs(x,xletter,yletter,αo_y):
    nby = len(αo_y) - 1
    xsprefs = xletter + str(x) +' : ' 
    for y in range(nby):
        ind = np.where(αo_y== (nby - y ))[0][0]
        if (ind == nby):
            break        
        xsprefs = xsprefs + yletter + str( ind ) + ' > '
    return(xsprefs[:-3])

def print_prefs(self,xs=[],ys=[]):
    if xs == [] and ys==[] :
        xs = range(self.nbx)
        ys = range(self.nby)
    if xs != [] :
        for x in xs :
            print(list_prefs(x,'x','y',self.αo_x_y[x,:]))
        
    if xs != [] and ys != [] :
        print('===')
        
    if ys != [] :
        for y in ys :
            print(list_prefs(y,'y','x',self.γo_x_y[:,y]))

NTU_market.print_prefs = print_prefs

We simulate such a market and print the corresponding preferences:

In [4]:
np.random.seed(seed=1000)
running_mkt = NTU_market(np.random.rand(3,2)-0.1,np.random.rand(3,2)-0.2)
running_mkt.print_prefs()

x0 : y0 > y1
x1 : y0 > y1
x2 : y0 > y1
===
y0 : x1 > x2
y1 : x1 > x2 > x0


The following example is taken from [RS90], example 2.17

In [5]:
# example 2.17 in [RS90]
α_x_y = np.array([[0,1,2,3],[1,0,3,2],[2,3,0,1],[3,2,1,0]])
γ_x_y = np.array([[3,2,1,0],[2,3,0,1],[1,0,3,2],[0,1,2,3]])
rs_ex_2_17 =  NTU_market(α_x_y,γ_x_y)
rs_ex_2_17.print_prefs()

x0 : y3 > y2 > y1 > y0
x1 : y2 > y3 > y0 > y1
x2 : y1 > y0 > y3 > y2
x3 : y0 > y1 > y2 > y3
===
y0 : x0 > x1 > x2 > x3
y1 : x1 > x0 > x3 > x2
y2 : x2 > x3 > x0 > x1
y3 : x3 > x2 > x1 > x0


# Stable matchings

We shall see two notions of stable matchings with non-transferable utility (NTU):<br> 
* First, the historical notion (due to Gale and Shapley) of *Gale-Shapley NTU stable matchings*. 
* Next, a related notion of stability that will fit into the framework we saw yesteray, which we shall call *aggregate NTU stable matchings*, following Galichon and Hsieh (2019).

## Stability in the Gale-Shapley sense

**Assume that there is one individual per type:**<br>
$n_x = 1$ and $m_y = 1$ for all $x\in\mathcal{X}$ and $y\in\mathcal{Y}$.<br>

As before, $\mu_{xy}\in\{0,1\}$ will be a dummy variable equal to 1 iff $x$ is matched with $y$,<br>
    and $u_x$ and $v_y$ are the payoffs of worker $x$ and firm $y$ at equilibrium.

**Definition.** $(\mu,u,v)$ is a stable outcome in the Gale-Shapley sense if:

(i) Population constraints are satisfied<br>
$\left\{
\begin{array}[l]
~\sum_{y} \mu_{xy} + \mu_{x0} = n_{x}=1 \\
\sum_{x}\mu_{xy} + \mu_{0y} = m_{y}=1
\end{array}
\right.$<br>
    
(ii) There is no blocking pair and all individuals are rational:<br>
$ \max \{ u_x - \alpha_{xy},v_y - \gamma_{xy} \} \geq 0$, and<br>
$u_x \geq 0$ and $v_y \geq 0$

 
(iii) Strong complementarity:<br>
$\mu_{xy}>0$ implies $u_x = \alpha_{xy}$ and $v_y = \gamma_{xy}$<br>
$\mu_{x0}>0$ implies $u_x = 0$<br>
$\mu_{0y}>0$ implies $v_y = 0$

### Remark

* In the Gale-Shapley definition:<br>
$\mu_{xy}>0$ implies $u_x = \alpha_{xy}$ and $v_y = \gamma_{xy}$<br>

* This is stronger than assuming<br>
$\mu_{xy}>0$ implies $\max \{ u_x - \alpha_{xy},v_y - \gamma_{xy} \} \geq 0$<br>
Indeed, in the latter case, $(u_x,v_y)$ needs to be on the NTU frontier but doesn't have to be the efficient point $(\alpha_{xy},\gamma_{xy})$ as required in Gale-Shapley.

* Galichon-Hsieh impose the latter condition as the definition of stability, which is weaker than Gale-Shapley. We shall see later why it may make sense not to impose the outcome to be Pareto efficient. 


The following function detects stable matchings.

In [6]:
def is_stable(self, μ_x_y = None, output=0 ):
    if (μ_x_y is None):
        μ_x_y = self.eq_μ_x_y
    μext_x_y0 = np.hstack([μ_x_y, 1-np.sum(μ_x_y,axis = 1).reshape(-1,1) ])
    μext_x0_y = np.vstack([μ_x_y, 1-np.sum(μ_x_y,axis = 0).reshape(1,-1) ])
    if (np.logical_and(μext_x_y0 != 0  , μext_x_y0 != 1 )).any() or (np.logical_and(μext_x0_y != 0  , μext_x0_y != 1 )).any():
        if output > 0 :
            print('The μ is not feasible.')
        return(False)
    uo_x = np.sum(μext_x_y0 * self.αo_x_y, axis = 1)
    vo_y = np.sum(μext_x0_y * self.γo_x_y, axis = 0)
    for x in range(self.nbx):
        for y in range(self.nby):
            if (self.αo_x_y[x,y] > uo_x[x]) and (self.γo_x_y[x,y] > vo_y[y]):
                if output > 0 :
                    print('The matching is not stable. One blocking pair is: x'+str(x)+',y'+str(y)+'.')
                return(False)
    # end for x and y
    if output > 0 :
        print('The matching is stable.')
    return (True)

NTU_market.is_stable = is_stable

Let's go back to example 2.17 in [RS90], and let's check the stability of the following matching: 

In [7]:
μ_x_y = np.array([[0,0,0,1],[1,0,0,0],[0,1,0,0],[0,0,1,0]])
rs_ex_2_17.is_stable(μ_x_y,output=1)

The matching is not stable. One blocking pair is: x1,y3.


False

In [8]:
tst = np.array([[0,1,2],[3,4,5]])
tst[1,range(2)]=-1
tst

array([[ 0,  1,  2],
       [-1, -1,  5]])

In [9]:
tst = np.array([[0,1,2],[3,4,5]])
tst[1,0:2]=-1
tst

array([[ 0,  1,  2],
       [-1, -1,  5]])

Let's try again with the following alternative matching:

In [10]:
μ_x_y = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
rs_ex_2_17.is_stable(μ_x_y,output=1)

The matching is stable.


True

## Deferred acceptance: Gale and Shapley's algorithm


The reference for this section is [GS62].

**Principle:** Workers make offers to firms that have not rejected them yet. Define:

* $A^{t} \subseteq \mathcal{X} \times \mathcal{Y} =$set of available firms to worker at
time $t$

* $P^{t} \subseteq \mathcal{X} \times \mathcal{Y} =$set of proposals made by workers at
time $t$ 

* $E^{t}\subseteq \mathcal{X} \times \mathcal{Y} =$set of proposals kept by firms at the
end of round $t$ 

For  $B \subseteq \mathcal{X} \times \mathcal{Y}$, we introduce the following notations:

* for $x \in \mathcal{X}$, denote $B(x)$ the set of $y \in \mathcal{Y}$ such that $xy \in B$.

* for $y \in \mathcal{Y}$, denote $B(y)$ the set of $x \in \mathcal{X}$ such that $xy \in B$.


**Deferred acceptance algorithm of Gale and Shapley.** 

At time $t=0$, all firms are avaiable to anyone $A^{0} =\mathcal{X} \times \mathcal{Y}$.

Iterate over $t$:

$
\left\{
\begin{array}{l}
P^{t}=\left\{ xy\in \mathcal{X}\times \mathcal{Y}:y\in \arg \max_{y\in
A^{t}\left( x\right) \cup \left\{ 0\right\} }\left\{ \alpha _{xy}\right\}
\right\} \\
E^{t}=\left\{ xy\in \mathcal{X}\times \mathcal{Y}:x\in \arg \max_{x\in
P^{t}\left( y\right) \cup \left\{ 0\right\} }\left\{ \gamma _{xy}\right\}
\right\} \\
A^{t+1} =A^{t} \backslash \left\{ P^{t} \backslash  E^{t} \right\} 
\end{array}
\right.
$



Repeat until $P^{t} =E^{t} $, i.e. no offer is rejected.

Let's implement as follows:

In [11]:
def solveGaleShapley(self,output=0, trace=False):
    if (output>=2):
        print("Offers made and kept are denoted +1; offers made and rejected are denoted -1.")
    self.comp_nbsteps = 0
    tracemax = self.nbx*self.nby
    if trace:
        self.traceu0_x_t = np.zeros((tracemax,self.nbx))    
    μA_x_y = np.ones((self.nbx+1, self.nby+1), dtype = 'int64') # initially all offers are non rejected
    while True :
        μP_x_y = np.zeros((self.nbx+1, self.nby+1), dtype = 'int64')
        props_x = np.ma.masked_array(self.αo_x_y, μA_x_y[0:-1,:] ==0).argmax(axis = 1) # x's makes an offer to their favorite y
        μP_x_y[range(self.nbx),props_x] = 1 
        μP_x_y[self.nbx,0:self.nby] = 1
        
        
        μE_x_y = np.zeros((self.nbx+1, self.nby+1), dtype = 'int64') # y's retains their favorite offer among those made:
        rets_y = np.ma.masked_array(self.γo_x_y, μP_x_y[:,0:-1] ==0).argmax(axis = 0)
        μE_x_y[rets_y,range(self.nby)] = 1

        rej_x_y = μP_x_y - μE_x_y # compute rejected offers
        rej_x_y[self.nbx,:] = 0
        rej_x_y[:,self.nby] = 0
        

        μA_x_y = μA_x_y - rej_x_y  # offers from x that have been rejected are no longer available to x
        if output >= 2:
            print("Round "+str(self.comp_nbsteps)+":\n" , ( (2*μE_x_y-1) * μP_x_y)[0:-1,0:-1])
        if trace and self.comp_nbsteps<tracemax:
            for x in range(self.nbx):
                self.traceu0_x_t[self.comp_nbsteps,x] = np.sum(self.αo_x_y [x,:] * μP_x_y[x,:])
        self.comp_nbsteps +=1
        self.eq_μ_x_y = μE_x_y[0:-1,0:-1]
        if np.max(np.abs(rej_x_y)) == 0: 
            if trace:
                self.traceu0_x_t = self.traceu0_x_t[0:min(self.comp_nbsteps,tracemax),:]
            break # if all offers have been accepted, then algorithm stops
    return (0)

NTU_market.solveGaleShapley = solveGaleShapley 


Test on our example:

In [12]:
running_mkt.solveGaleShapley(output=2,trace=True)
running_mkt.is_stable()

Offers made and kept are denoted +1; offers made and rejected are denoted -1.
Round 0:
 [[-1  0]
 [ 1  0]
 [-1  0]]
Round 1:
 [[ 0 -1]
 [ 1  0]
 [ 0  1]]
Round 2:
 [[0 0]
 [1 0]
 [0 1]]


True

We investigate a bit more systematically:

In [13]:
np.random.seed(seed=77)
nbx,nby=50,35
for i in range(10):
    tstmkt = NTU_market(np.random.rand(nbx,nby)-0.3,np.random.rand(nbx,nby)-0.2)
    tstmkt.solveGaleShapley()
    print('Stable = '+str(tstmkt.is_stable())+'; nb of steps = '+str(tstmkt.comp_nbsteps))


Stable = True; nb of steps = 71
Stable = True; nb of steps = 73
Stable = True; nb of steps = 83
Stable = True; nb of steps = 83
Stable = True; nb of steps = 93
Stable = True; nb of steps = 96
Stable = True; nb of steps = 78
Stable = True; nb of steps = 63
Stable = True; nb of steps = 69
Stable = True; nb of steps = 105


## Deferred acceptance revisited: Adachi's algorithm

The reference for this section is [A00].

**Adachi's algorithm.**

Set initially $v^0_y$: say $v^0_y < \min_{x\in \mathcal{X}_0} \left\{ \gamma_{xy} \right\}$.

Iterate over t:

$\left\{ 
\begin{array}{l}
u_{x}^{t+1}=\max \left\{ \max_{y\in \mathcal{Y}}\left\{ \alpha _{xy}:\gamma
_{xy}\geq v_{y}^{t}\right\} ,\alpha _{x0}\right\}  \\ 
v_{y}^{t+1}=\max \left\{ \max_{x\in \mathcal{X}}\left\{ \gamma _{xy}:\alpha
_{xy}\geq u_{x}^{t+1}\right\} ,\gamma _{0y}\right\}
\end{array}
\right. $

until $u^{t+1} = u^{t}$. 

We now implement Adachi's algorithm. It will be based on a pair of functions `u_from_v` and `v_from_u` which implement the maximization problem above, namely:<br>
$\max \left\{ \max_{y\in \mathcal{Y}}\left\{ \alpha _{xy}:\gamma_{xy}\geq v_{y}\right\} ,\alpha _{x0}\right\}$, and<br>
$\max \left\{ \max_{x\in \mathcal{X}}\left\{ \gamma _{xy}:\alpha
_{xy}\geq u_{x}\right\} ,\gamma _{0y}\right\}$.


In [14]:
def u_from_v(self,vo_y):
    excluded = np.hstack([(self.γo_x_y < vo_y)[0:-1,:],np.zeros((self.nbx,1))])
    return(np.ma.masked_array(self.αo_x_y,  excluded ).max(axis = 1).data)

NTU_market.u_from_v = u_from_v

def  v_from_u(self,uo_x):
    excluded = np.vstack([(self.αo_x_y < uo_x.reshape((-1,1)))[:,0:-1],np.zeros((1,self.nby))])
    return(np.ma.masked_array(self.γo_x_y,  excluded ).max(axis = 0).data)

NTU_market.v_from_u = v_from_u

def μ_from_u(self, uo_x):
    return np.where( (self.αo_x_y == uo_x.reshape((-1,1))) , 1 , 0 )[:,0:-1]

NTU_market.μ_from_u = μ_from_u

def μ_from_v(self, vo_y):
    return np.where( self.γo_x_y == vo_y , 1 , 0 )[0:-1,:]

NTU_market.μ_from_v = μ_from_v


Adachi's algorithm iterates the loop above:

In [15]:
def solveAdachi(self,output=0,trace=False, startu_x = None, startv_y = None ):
    self.comp_nbsteps = 0
    tracemax = self.nbx*self.nby
    if trace:
        self.traceu0_x_t = np.zeros((tracemax,self.nbx))    
    if startu_x is None:
        uo_x = self.largex * np.ones(self.nbx, dtype = 'int64') # x's utilities are highest
    else:
        uo_x = startu_x
    if startv_y is None:
        vo_y = self.smally * np.ones(self.nby, dtype = 'int64') # y's utilities are lowest
    else:
        vo_y = startv_y
    while True :
        uonew_x = self.u_from_v(vo_y) # each x proposes to favorite y among those willing to consider them
        if (uonew_x == uo_x).all() :
            if trace:
                self.traceu0_x_t = self.traceu0_x_t[0:min(self.comp_nbsteps,tracemax),:]
            break
        uo_x = uonew_x      
        vo_y = self.v_from_u(uo_x) # each y proposes to favorite x among those willing to consider them
        if output >= 2:
            μP_x_y = self.μ_from_u(uo_x)
            μE_x_y = self.μ_from_v(vo_y)
            print("Round "+str(self.comp_nbsteps)+":\n","μ_P=\n" ,μP_x_y,"\n μ_E=\n", μE_x_y)
        if trace and self.comp_nbsteps<tracemax:
            self.traceu0_x_t[self.comp_nbsteps,:] = uo_x
        self.comp_nbsteps += 1
    self.eq_μ_x_y = self.μ_from_v(vo_y)
    return(0)

NTU_market.solveAdachi = solveAdachi 

We try it on `running_mkt`:

In [16]:
running_mkt.solveAdachi()
running_mkt.is_stable()

True

Next, we investigate a bit more Adachi's algorithm similarly to what we did above with Gale and Shapley's algorithm, and in comparison with the latter:

In [17]:
print("Times: Gale-Shapley; Adachi")

np.random.seed(seed=77)
nbx,nby=50,35
for i in range(10):
    tstmkt = NTU_market(np.random.rand(nbx,nby)-0.3,np.random.rand(nbx,nby)-0.2)
    res=tstmkt.solveGaleShapley()
    if not tstmkt.is_stable() :
        raise Exception('Output of Gale Shapley is not stable')
    i1 = tstmkt.comp_nbsteps
    t1 = tstmkt.comp_time
    mu1 = tstmkt.eq_μ_x_y
    res=tstmkt.solveAdachi()
    i2 = tstmkt.comp_nbsteps
    t2 = tstmkt.comp_time
    mu2 = tstmkt.eq_μ_x_y
    if not tstmkt.is_stable() :
        raise Exception('Output of Adachi is not stable')
    if  np.max(np.abs(mu1 - mu2)) != 0:
        raise Exception(seed,': results differ.')
    print("GS steps = "+ str(i1)+"\t; A steps = "+str(i2))
print('All matching outcomes from Gale-Shapley and Adachi are stable and coincide.')


Times: Gale-Shapley; Adachi
GS steps = 71	; A steps = 10
GS steps = 73	; A steps = 12
GS steps = 83	; A steps = 10
GS steps = 83	; A steps = 10
GS steps = 93	; A steps = 12
GS steps = 96	; A steps = 13
GS steps = 78	; A steps = 11
GS steps = 63	; A steps = 10
GS steps = 69	; A steps = 11
GS steps = 105	; A steps = 14
All matching outcomes from Gale-Shapley and Adachi are stable and coincide.


# Testing an alternative representation of Adachi

In this section, we show experimentally that Adachi's algorithm can equivalently be expressed by the following formulation:

**Adachi Bis algorithm.**

Set initially $v^0_y$: say $v^0_y < \min_{x\in \mathcal{X}_0} \left\{ \gamma_{xy} \right\}$.

Iterate over t:

$\left\{ 
\begin{array}{l}
u_{x}^{t+1}=\max \left\{ \max_{y\in \mathcal{Y}}\left\{ \alpha _{xy}:\gamma
_{xy}\geq v_{y}^{t}\right\} ,\alpha _{x0}\right\}  \\ 
v_{y}^{t+1}=\max \left\{ \max_{x\in \mathcal{X}}\left\{ \gamma _{xy}:\alpha
_{xy}= u_{x}^{t+1}\right\} ,\gamma _{0y}\right\}
\end{array}
\right. $

until $u^{t+1} = u^{t}$. 

In [18]:
def  v_from_u_bis(self,uo_x):
    excluded = np.vstack([(self.αo_x_y != uo_x.reshape((-1,1)))[:,0:-1],np.zeros((1,self.nby))])
    return(np.ma.masked_array(self.γo_x_y,  excluded ).max(axis = 0).data)

NTU_market.v_from_u_bis = v_from_u_bis

In [19]:
def solveAdachiBis(self,output=0,trace=False):
    self.comp_nbsteps = 0
    tracemax = self.nbx*self.nby
    if trace:
        self.traceu0_x_t = np.zeros((tracemax,self.nbx))    
    uo_x = self.largex * np.ones(self.nbx, dtype = 'int64') # x's utilities are highest
    vo_y = self.smally * np.ones(self.nby, dtype = 'int64') # y's utilities are lowest
    while True :
        uonew_x = self.u_from_v(vo_y) # each x proposes to favorite y among those willing to consider them
        if (uonew_x == uo_x).all() :
            if trace:
                self.traceu0_x_t = self.traceu0_x_t[0:min(self.comp_nbsteps,tracemax),:]
            break
        uo_x = uonew_x      
        vo_y = self.v_from_u_bis(uo_x) # each y proposes to favorite x among those willing to consider them
        if trace and self.comp_nbsteps<tracemax:
            self.traceu0_x_t[self.comp_nbsteps,:] = uo_x
        self.comp_nbsteps += 1
    self.eq_μ_x_y = self.μ_from_v(vo_y)
    return(0)

NTU_market.solveAdachiBis = solveAdachiBis 

In [20]:
np.random.seed(seed=77)
nbx,nby=50,35
for i in range(10):
    mkt_tst = NTU_market(np.random.rand(4,3)-0.2,np.random.rand(4,3)-0.3)
    mkt_tst.solveAdachi(trace =True)
    t1  = mkt_tst.traceu0_x_t
    mkt_tst.solveAdachiBis(trace=True)
    t2 = mkt_tst.traceu0_x_t
    print("Discrepancy between Adachi and Adachi Bis=", np.max(np.abs(t2-t1)))

Discrepancy between Adachi and Adachi Bis= 0.0
Discrepancy between Adachi and Adachi Bis= 0.0
Discrepancy between Adachi and Adachi Bis= 0.0
Discrepancy between Adachi and Adachi Bis= 0.0
Discrepancy between Adachi and Adachi Bis= 0.0
Discrepancy between Adachi and Adachi Bis= 0.0
Discrepancy between Adachi and Adachi Bis= 0.0
Discrepancy between Adachi and Adachi Bis= 0.0
Discrepancy between Adachi and Adachi Bis= 0.0
Discrepancy between Adachi and Adachi Bis= 0.0


## Reformulating Adachi and Gale-Shapley coordinate updates algorithm

As usual, change the sign of $u_{x}$ take $p=\left( u,-v\right) $ 
and define<br>
$\left\{ 
\begin{array}{l}
Q_{x}\left( p\right) =p_{x}-\max_{y\in \mathcal{Y}}\left\{ \alpha
_{xy}:p_{y}\geq -\gamma _{xy},0\right\} ,x\in \mathcal{X} \\ 
Q_{y}\left( p\right) =p_{y}-\min_{x\in \mathcal{X}}\left\{ -\gamma
_{xy}:\alpha _{xy}\geq p_{x},0\right\} ,y\in \mathcal{Y}%
\end{array}%
\right.$

In [21]:
def Q_z(self,p_z):
    uo_x = p_z[0:self.nbx]
    vo_y = - p_z[self.nbx:(self.nbx+self.nby)]
    uonew_x = self.u_from_v(vo_y)
    vonew_y = self.v_from_u(uo_x)
    return(np.append(uo_x - uonew_x,vonew_y - vo_y))

NTU_market.Q_z = Q_z    

## Adachi as Gauss-Seidel

Start by $p_{x}^{0}=\max_{y\in \mathcal{Y}_{0}}\left\{ \alpha _{xy}\right\} $
and $p_{y}^{0}=\max_{x\in \mathcal{X}_{0}}\left\{ -\gamma _{xy}\right\} $,
which are such that $e\left( p\right) \geq 0$.

Adachi reinteprets as a blockwise Gauss-Seidel algorithm<br>
$\left\{ 
\begin{array}{l}
p_{x}^{t+1}:Q_{x}\left( p_{x}^{t+1},\left( p_{y}^{t}\right) _{y}\right) =0,
\\ 
p_{y}^{t+1}:Q_{y}\left( p_{y}^{t+1},\left( p_{x}^{t+1}\right) _{x}\right) =0.%
\end{array}%
\right.$

In [22]:
def cux_z(self,p_z):
    uo_x = p_z[0:self.nbx]
    vo_y = - p_z[self.nbx:(self.nbx+self.nby)]
    uonew_x = self.u_from_v(vo_y)
    return (np.append(uonew_x , - vo_y))
        
NTU_market.cux_z = cux_z


def cuy_z(self,p_z):
    uo_x = p_z[0:self.nbx]
    vo_y = - p_z[self.nbx:(self.nbx+self.nby)]
    excluded = np.vstack([(self.αo_x_y < np.repeat([uo_x],self.nby+1,axis = 1).reshape((self.nbx,-1)))[:,0:-1],np.repeat(False,self.nby).reshape((-1,self.nby))])
    vonew_y = self.v_from_u(uo_x)
    return( np.append(uo_x, - vonew_y ) )

NTU_market.cuy_z = cuy_z

In [23]:
def solveCU(self,output=0,trace=False):
    self.comp_nbsteps = 0
    tracemax = self.nbx*self.nby
    if trace:
        self.traceu0_x_t = np.zeros((tracemax,self.nbx))    
    p_z = np.append(self.largex * np.ones(self.nbx), - self.smally * np.ones(self.nby) )
    while True :
        pnew_z = self.cux_z(p_z) 
        pnew_z = self.cuy_z(pnew_z)
        if (pnew_z == p_z).all() :
            if trace:
                self.traceu0_x_t = self.traceu0_x_t[0:min(self.comp_nbsteps,tracemax),:]
            break
        p_z = pnew_z      
        if trace and self.comp_nbsteps<tracemax:
            self.traceu0_x_t[self.comp_nbsteps,:] = p_z[0:self.nbx]
        self.comp_nbsteps += 1
    self.eq_μ_x_y = self.μ_from_v(- p_z[self.nbx:(self.nbx+self.nby)])
    return(0)

NTU_market.solveCU = solveCU

We test numerically that Adachi and Gauss-Seidel coincide:

In [24]:
np.random.seed(1000)

mkt_tst = NTU_market(np.random.rand(80,120)-0.2,np.random.rand(80,120)-0.3)

mkt_tst.solveAdachi(trace =True)
t1  = mkt_tst.traceu0_x_t

mkt_tst.solveCU(trace=True)
t2 = mkt_tst.traceu0_x_t

print("Discrepancy between Adachi and Gauss-Seidel=", np.max(t2-t1))


Discrepancy between Adachi and Gauss-Seidel= 0.0


## Gale-Shapley as damped Gauss-Seidel

Gale and Shapley interpets as<br>
$\left\{ 
\begin{array}{l}
p_{x}^{t+1}=decr_{x}\left( p_{x}^{t}\right) \text{ if }Q_{x}\left(
p_{x}^{t+1},\left( p_{y}^{t}\right) _{y}\right) >0\text{, } \\ 
p_{x}^{t+1}=incr_{x}\left( p_{x}^{t}\right) \text{ if }Q_{x}\left(
p_{x}^{t+1},\left( p_{y}^{t}\right) _{y}\right) <0 \\ 
p_{x}^{t+1}=p_{x}^{t}\text{ if }Q_{x}\left( p_{x}^{t+1},\left(
p_{y}^{t}\right) _{y}\right) =0, \\ 
p_{y}^{t+1}:Q_{y}\left( p_{y}^{t+1},\left( p_{x}^{t+1}\right) _{x}\right) =0%
\end{array}%
\right.$<br> 
where<br>
$decr_{x}\left( p\right) =\max_{y\in \mathcal{Y}_{0}}\left\{ \alpha
_{xy}:\alpha _{xy}<p\right\} $ is the next value below $p$ (in terms of the $\alpha _{xy}$'s), and<br> 
$incr_{x}\left(p\right) =\min_{x\in \mathcal{X}_{0}}\left\{ \alpha _{xy}:\alpha
_{xy}>p\right\} $ is the next value above $p$.



In [25]:
def damped_cux_z(self,p_z):
    uo_x = p_z[0:self.nbx]
    vo_y = - p_z[self.nbx:(self.nbx+self.nby)]
    uonew_x = self.u_from_v(vo_y)
    return (np.append(uo_x - np.where(uo_x > uonew_x, 1,0)+ np.where(uo_x < uonew_x, 1,0) , - vo_y))
        
NTU_market.damped_cux_z = damped_cux_z


In [26]:
def solveDampedCU(self,output=0,trace=False):
    self.comp_nbsteps = 0
    tracemax = self.nbx*self.nby
    if trace:
        self.traceu0_x_t = np.zeros((tracemax,self.nbx))    
    p_z = np.append(self.largex * np.ones(self.nbx), - self.smally * np.ones(self.nby) )
    while True :
        pnew_z = self.damped_cux_z(p_z)   # each x proposes to favorite y among those willing to consider them:
        pnew_z = self.cuy_z(pnew_z)
        if (pnew_z == p_z).all() :
            if trace:
                self.traceu0_x_t = self.traceu0_x_t[0:min(self.comp_nbsteps,tracemax),:]
            break
        p_z = pnew_z      
        # each y proposes to favorite x among those willing to consider them:
        if trace and self.comp_nbsteps<tracemax:
            self.traceu0_x_t[self.comp_nbsteps,:] = p_z[0:self.nbx]
        self.comp_nbsteps += 1
    self.eq_μ_x_y = self.μ_from_v(- p_z[self.nbx:(self.nbx+self.nby)])
    return(0)

NTU_market.solveDampedCU = solveDampedCU     


We test numerically that Gale and Shapley and damped Gauss-Seidel coincide:

In [27]:
np.random.seed(1000)

mkt_tst = NTU_market(np.random.rand(80,120)-0.2,np.random.rand(80,120)-0.3)

mkt_tst.solveGaleShapley(trace =True)
t1  = mkt_tst.traceu0_x_t
#print(mkt_tst.comp)

mkt_tst.solveDampedCU(trace=True)
t2 = mkt_tst.traceu0_x_t

print("Discrepancy between Gale and Shapley and coordinate update=", np.max(t2-t1))


Discrepancy between Gale and Shapley and coordinate update= 0.0


## Comparing running times

We build a wrapper that keeps track of the times.


In [28]:
from time import time

def solveDeferredAcceptance(self, algorithm = 'Adachi', output=0, trace=False):
    start_time = time()
    if algorithm == 'GS' :
        self.solveGaleShapley(output,trace)
    elif algorithm == 'Adachi' :
        self.solveAdachi(output,trace)
    elif algorithm == 'DARUM' :
        self.solveDARUM(output,trace)   
    elif algorithm == 'CU' :
        self.solveCU(output,trace)
    else:
        raise Exception("Algorithm " + algorithm + " is not implemented.")
    self.comp_time =  time() - start_time
    if output >= 1:
        print("Converged in ",self.comp_nbsteps," steps and ", self.comp_time, " seconds.")
    if output == 1:
        print("mu_x_y=",self.eq_μ_x_y) 
    return (0)

    
NTU_market.solveDeferredAcceptance = solveDeferredAcceptance

## Enumeration of stable matchings

Cf ROTH-SOTOMAYOR P. 62

In [29]:
def matched_pairs(self, μ_x_y = None):
    if μ_x_y is None:
        μ_x_y = self.eq_μ_x_y
    nzx,nzy = np.nonzero(μ_x_y)
    return [(nzx[i],nzy[i]) for i in range(len(nzx) )]

NTU_market.matched_pairs = matched_pairs

In [61]:
def next_us(self,uo_x,down = True):
    μ_x_y = self.μ_from_u(uo_x)
    if down:
        incr= -1
    else:
        incr = 1
    g = nx.DiGraph()
    xs = list(range(self.nbx))
    ys = list(range(self.nbx,self.nby))
    g.add_nodes_from(xs+ys)
    g.add_edges_from([(x,self.nbx+y) for x,y in self.matched_pairs(μ_x_y)])
    μprime_x_y = np.where( (self.αo_x_y.transpose() == uo_x + incr).transpose() , 1 , 0 )[:,0:-1]
    compatible_pairs = self.matched_pairs(μprime_x_y)
    g.add_edges_from([(self.nbx+j,i) for (i,j) in compatible_pairs])
    cycles = list(nx.simple_cycles(g))
    newus = []
    for c in cycles:
        xs_to_update = [x for x in c if (x <self.nbx)]
        nextu_x = uo_x.copy()
        nextu_x[xs_to_update] += incr
        newus.append(nextu_x)
    return(newus)

NTU_market.next_us = next_us

In [62]:
rs_ex_2_17.solveGaleShapley()
uo_x = np.sum(rs_ex_2_17.αo_x_y [:,0:-1] * rs_ex_2_17.eq_μ_x_y,axis = 1)
print(rs_ex_2_17.next_us(uo_x))

[array([4, 4, 3, 3], dtype=int64), array([3, 3, 4, 4], dtype=int64)]


In [63]:
def enumerate_us(self):
    self.solveGaleShapley()
    uo_x = np.sum(self.αo_x_y [:,0:-1] * self.eq_μ_x_y,axis = 1)
    # H = nx.DiGraph()
    us_list = [uo_x.tolist()]
    i = 0
    # H.add_nodes_from([i])
    # H.nodes[i]['pos'] = (0,0)
    while (i < len(us_list)):
        inds_alreadyhere = []
        us_toadd = [arr.tolist() for arr in self.next_us(np.array(us_list[i],dtype='int64'))]
        for u_x in us_toadd:
            if u_x in us_list:
                inds_alreadyhere.append(us_list.index(u_x))
                us_toadd.remove(u_x)
        
        inds_toadd = list(range(len(us_list),len(us_list)+len(us_toadd) ))
        # H.add_nodes_from(inds_toadd)
        # for node in inds_toadd:
            # H.nodes[node]['pos'] = (0,H.nodes[i]['pos'][1]+1)
        #H.add_edges_from([(i,j) for j in inds_alreadyhere+inds_toadd])
        us_list = us_list + us_toadd 
        i += 1
    return([np.array(u,dtype = 'int64') for u in us_list])

NTU_market.enumerate_us = enumerate_us

In [64]:
the_u_s = rs_ex_2_17.enumerate_us()
the_u_s

[array([4, 4, 4, 4], dtype=int64),
 array([4, 4, 3, 3], dtype=int64),
 array([3, 3, 4, 4], dtype=int64),
 array([3, 3, 3, 3], dtype=int64),
 array([3, 2, 2, 3], dtype=int64),
 array([2, 3, 3, 2], dtype=int64),
 array([2, 2, 2, 2], dtype=int64),
 array([2, 2, 1, 1], dtype=int64),
 array([1, 1, 2, 2], dtype=int64),
 array([1, 1, 1, 1], dtype=int64)]

In [65]:
for u_x in the_u_s:
    μ_x_y = rs_ex_2_17.μ_from_u(u_x)
    print(rs_ex_2_17.is_stable(μ_x_y))

True
True
True
True
True
True
True
True
True
True


# Aggregate stable matchings

The reference for this section is [GH19].


In [37]:
def proposeNoHet(self,axis,μbar_x_y):
    if axis == 0 : # if proposing side = x
        n_x = self.n_x
        nbx = self.nbx
        nby = self.nby
        prefs_x_y = self.prefslistα_x_y
    else:
        n_x = self.m_y
        nbx = self.nby
        nby = self.nbx
        prefs_x_y = self.prefslistγ_x_y.transpose()    
    μ_x_y = np.zeros((nbx,nby))
    for x in range(nbx):
        nxres = n_x[x]
        for yind in prefs_x_y[x,]:
            if μbar_x_y[x,yind] > 0:
                μ_x_y[x,yind] = min(nxres , μbar_x_y[x,yind])
                nxres -= μ_x_y[x,yind]
            if nxres == 0:
                break
    return(μ_x_y)

NTU_market.proposeNoHet = proposeNoHet

def propose(self,axis,μbar_x_y,heterogeneity = 'none'):
    if heterogeneity == 'none':
        μ_x_y = self.proposeNoHet(axis,μbar_x_y)
    else:
        raise Exception("Heterogeneity " + heterogeneity + " is not supported.")
    return (μ_x_y)

NTU_market.propose = propose

def solveDARUM(self,output=0,trace=False,tol = 1e-5):
    self.comp_nbsteps = 0
    tracemax = self.nbx*self.nby
    if trace:
        self.traceu0_x_t = np.zeros((self.nbx,tracemax))    
    μA_x_y = np.zeros((self.nbx, self.nby)) # initially all offers are non rejected
    for x in range(self.nbx):
        for y in range(self.nby):
            μA_x_y[x,y] = max(self.n_x[x],self.m_y[y])
    while True :
        μP_x_y = self.propose(0 , μA_x_y,'none') # the x's pick their preferred offers among those not rejected
        μE_x_y = self.propose(1, μP_x_y.transpose(),'none').transpose() # the y's pick their preferred offers among those made
        rej_x_y = μP_x_y - μE_x_y # compute rejected offers
        if np.max(np.abs(rej_x_y)) < tol: 
            if trace:
                self.traceu0_x_t = self.traceu0_x_t[:,0:min(self.comp_nbsteps,tracemax)]
            break # if all offers have been accepted (within tolerange), then algorithm stops
        μA_x_y = μA_x_y - rej_x_y  # offers from x that have been rejected are no longer available to x
        if output >= 2:
            print("mu_P=, mu_E=" )
            print(μP_x_y)
            print(μE_x_y)        
        if trace and self.comp_nbsteps<tracemax:
            for x in range(self.nbx):
                self.traceu0_x_t[x,self.comp_nbsteps] = np.sum(self.αo_x_y [x,:] * μP_x_y[x,:])

        self.comp_nbsteps +=1
    self.eq_μ_x_y = μE_x_y
    return (0)

NTU_market.solveDARUM = solveDARUM 